In [2]:
import os,torch,h5py
import time
import numpy as np

In [3]:
def trans2h5(pt_file_path):
    print(pt_file_path)
    data = torch.load(pt_file_path)
    hdf5_file_path = pt_file_path.replace('.pt','.h5')
    with h5py.File(hdf5_file_path, 'w') as h5_file:
        for key, value in data.items():
            print(key)
            if key in ['img','text']:
                value = value.astype('S80')
            elif key == 'ch_names':
                value =np.array(data['ch_names'],dtype='S4')
            if isinstance(value, torch.Tensor):
                h5_file.create_dataset(key, data=value.numpy())
            elif isinstance(value, (int, float, str, np.ndarray)):
                h5_file.create_dataset(key, data=value)
            else:
                raise TypeError(f"Unsupported data type: {type(value)}")
# trans2h5('/root/workspace/wht/multimodal_brain/datasets/things-eeg-small/Preprocessed_data_250Hz/sub-08/train.pt')
# trans2h5('/root/workspace/wht/multimodal_brain/datasets/things-eeg-small/Preprocessed_data_250Hz/sub-08/test.pt')

In [4]:
def test_speed(data_dir,read_type,repeat=500):
    for mode in ['train','test']:
        pt_path = os.path.join(data_dir,f'{mode}.pt')
        hdf5_file_path = pt_path.replace('.pt','.h5')
        if read_type=='pt':
            data = torch.load(pt_path)
        else:
            data = h5py.File(hdf5_file_path, 'r+')
        keys = list(data.keys())
        key='eeg'
        _ = data[key][:]
        length = len(_)
        indices = np.arange(length)
        np.random.shuffle(indices)
        for i in indices:
            d = _[i]

In [5]:
data_dirs =[]

#float64 87
#float32 15.5136,11.8792,19.3747
#float16 3.1612 2.9807 2.9229
#10 9.15 8.60
#29.1 30.0 244
data_dirs.append('/root/datasets/things-eeg-small/Preprocessed_data_250Hz')
data_dirs.append('/dev/shm/datasets/things-eeg-small/Preprocessed_data_250Hz')
data_dirs.append('/root/workspace/wht/multimodal_brain/datasets/things-eeg-small/Preprocessed_data_250Hz') 

for data_dir in data_dirs:
    start_time = time.time()
    for i in range(10):
        for mode in ['train','test']:
            pt_path = os.path.join(data_dir,'sub-08',f'{mode}.pt')
            data = torch.load(pt_path)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"代码运行时间: {elapsed_time:.4f} 秒")

代码运行时间: 30.2541 秒
代码运行时间: 29.6536 秒
代码运行时间: 31.7917 秒
